In [1]:
!git clone https://github.com/leoimpett/PyRetri-Colab &>/dev/null
%cd /content/PyRetri-Colab/
!pip install -r requirements.txt  &>/dev/null
!python  setup.py install &>/dev/null




import json
import requests
import tqdm
import multiprocessing

!mkdir ycba_images
!mkdir ycba_images/imgs

with open("/content/PyRetri-Colab/ycbaprints.json", "r") as myfile:
    data = json.load(myfile)
    

imkeys = []
imURLs = {}
imtitles = {}

for item in data["results"]["result"]:
    thiskey = item["binding"][2]["uri"]
    if thiskey not in imkeys:
        imkeys.append(thiskey)
        imURLs[thiskey] = item["binding"][1]["uri"]
        imtitles[thiskey] = item["binding"][0]["literal"]
        
def makeFilePath(imkey):
    endkey = imkey.split("/")[-1]
    return "ycba_images/imgs/" + endkey + ".jpg"


def downloadImage(thiskey):
    url = imURLs[thiskey]
    response = requests.get(url)
    if response.status_code == 200:
        with open( makeFilePath(thiskey) , 'wb') as f:
            f.write(response.content)
    return 0


pool = multiprocessing.Pool(16)

_ = pool.map(downloadImage, imkeys)


/content/PyRetri-Colab


# Extract features from the Gallery

In [2]:
images_data_path = "/content/PyRetri-Colab/ycbaprints/"
test_im = '/content/PyRetri-Colab/ycbaprints/imgs/2003.jpg'


from matplotlib import pyplot as plt
from skimage import io
import tqdm
import glob
import os

import torch

from pyretri.config import get_defaults_cfg, setup_cfg
from pyretri.datasets import build_folder, build_loader
from pyretri.models import build_model
from pyretri.extract import build_extract_helper

from torchvision import models

from pyretri.extract import make_data_json

make_data_json(images_data_path,"./data_jsons/gdrive.json","general")

# init and load retrieval pipeline settings
cfg = get_defaults_cfg()
cfg = setup_cfg(cfg, "configs/gdrive.yaml", [])

# build dataset and dataloader
dataset = build_folder("./data_jsons/gdrive.json", cfg.datasets)
dataloader = build_loader(dataset, cfg.datasets)

# build model
model = build_model(cfg.model)

# build helper and extract features
extract_helper = build_extract_helper(model, cfg.extract)
extract_helper.do_extract(dataloader, "./data/features/gdrive", save_interval=5000)




FileNotFoundError: ignored

# Now extract them from a test image

In [ ]:
!mkdir retrieved_images
!rm retrieved_images/*

import os
from PIL import Image
import numpy as np

from pyretri.config import get_defaults_cfg, setup_cfg
from pyretri.datasets import build_transformers
from pyretri.models import build_model
from pyretri.extract import build_extract_helper
from pyretri.index import build_index_helper, feature_loader



config_file = "configs/gdrive.yaml"


# init and load retrieval pipeline settings
cfg = get_defaults_cfg()
cfg = setup_cfg(cfg, config_file, [])

# set path for single image
path = test_im

# build transformers
transformers = build_transformers(cfg.datasets.transformers)

# build model
model = build_model(cfg.model)

# read image and convert it to tensor
img = Image.open(path).convert("RGB")
img_tensor = transformers(img)

# build helper and extract feature for single image
extract_helper = build_extract_helper(model, cfg.extract)
img_fea_info = extract_helper.do_single_extract(img_tensor)
stacked_feature = list()
for name in cfg.index.feature_names:
    assert name in img_fea_info[0], "invalid feature name: {} not in {}!".format(name, img_fea_info[0].keys())
    stacked_feature.append(img_fea_info[0][name].cpu())
img_fea = np.concatenate(stacked_feature, axis=1)

# load gallery features
gallery_fea, gallery_info, _ = feature_loader.load(cfg.index.gallery_fea_dir, cfg.index.feature_names)

# build helper and single index feature
index_helper = build_index_helper(cfg.index)


#index_result_info, query_fea, gallery_fea = index_helper.do_index(img_fea, img_fea_info, gallery_fea)

#index_helper.save_topk_retrieved_images('retrieved_images/', index_result_info[0], 5, gallery_info)

#print('single index have done!')



In [ ]:
index_result_info, query_fea, gallery_fea = index_helper.do_index(gallery_fea, gallery_info, gallery_fea)



In [ ]:
print(index_result_info[0]["ranked_neighbors_idx"])
print(len(index_result_info[0]["ranked_neighbors_idx"]))
print(len(index_result_info))


In [ ]:
import json

!rm jsonfiles/*
!mkdir jsonfiles

for element in gallery_info:
  elidx = element["idx"]
  filep = element["path"]
  filen = filep.split("/")[-1].split(".")[0]
  nearest = element["ranked_neighbors_idx"]
  
  nearData = []
  for jdx in nearest:
    nearData.append({"filename":gallery_info[jdx]["path"].split("/")[-1].split(".")[0],"similarity":1.0,"caption":"Test"})

  jsonfn = "jsonfiles/" + filen + ".json"
  with open(jsonfn, "w") as jfile:
    json.dump(nearData,jfile)


presentData = []
for element in gallery_info:
  presentData.append({"image":element["path"].split("/")[-1],"caption":"Test"})

print(len(presentData))
with open("randomImagePaths.json", "w") as jfile:
  json.dump(presentData,jfile)


!zip -r jsonfiles.zip jsonfiles

In [ ]:
plt.imshow(io.imread(test_im))

In [ ]:
imlist = glob.glob("./retrieved_images/*")
for iml in imlist:
  plt.figure()
  plt.imshow(io.imread(iml))